# Action Castle — FINAL PATCH (Complete)- Rose is guaranteed pickable (accepts `pick/take/get rose`).
- Helpful hint on `propose` if you lack the rose.
- Includes an auto-win demo cell.


In [1]:

import sys
from typing import Dict, List, Set, Callable, Optional

class Item:
    def __init__(self, name: str, aliases: List[str] = None, portable: bool = True, desc: str = ""):
        self.name = name.lower()
        self.aliases = set([self.name] + (aliases or []))
        self.portable = portable
        self.desc = desc
    def matches(self, token: str) -> bool:
        return token.lower() in self.aliases

class Location:
    def __init__(self, key: str, title: str, desc: str):
        self.key, self.title, self.desc = key, title, desc
        self.exits: Dict[str,str] = {}
        self.items: List[Item] = []
        self.blocks: List["Block"] = []

class Block:
    def __init__(self, from_key: str, direction: str, reason: str, condition: Callable[["Game"], bool]):
        self.from_key, self.direction, self.reason, self.condition = from_key, direction.lower(), reason, condition

class Action:
    def matches(self, raw: str, tokens: List[str]) -> bool: return False
    def run(self, game: "Game", raw: str, tokens: List[str]) -> str: return "Nothing happens."

class Game:
    def __init__(self):
        self.locations: Dict[str,Location] = {}
        self.items: Dict[str,Item] = {}
        self.player_loc: str = ""
        self.inventory: List[Item] = []
        self.flags: Set[str] = set()
        self.actions: List[Action] = []
        self.dead = False
        self.won = False

    def loc(self) -> Location: return self.locations[self.player_loc]
    def add_loc(self, key: str, title: str, desc: str) -> Location:
        L = Location(key, title, desc); self.locations[key] = L; return L
    def add_item(self, name: str, **kwargs) -> Item:
        it = Item(name, **kwargs); self.items[it.name] = it; return it
    def put_item(self, item_name: str, loc_key: str):
        self.locations[loc_key].items.append(self.items[item_name])
    def get_inventory_names(self) -> List[str]: return [it.name for it in self.inventory]
    def move(self, direction: str) -> str:
        d = direction.lower(); loc = self.loc()
        for block in loc.blocks:
            if block.direction == d and not block.condition(self):
                return block.reason
        if d in loc.exits:
            self.player_loc = loc.exits[d]; return self.look()
        return "You can't go that way."
    def take(self, token: str) -> str:
        token = token.lower(); here = self.loc()
        for it in list(here.items):
            if it.matches(token):
                if not it.portable: return f"You can't take the {it.name}."
                here.items.remove(it); self.inventory.append(it); return f"Taken {it.name}."
        return "You don't see that here."
    def drop(self, token: str) -> str:
        token = token.lower()
        for it in list(self.inventory):
            if it.matches(token):
                self.inventory.remove(it); self.loc().items.append(it); return f"Dropped {it.name}."
        return "You don't have that."
    def look(self) -> str:
        loc = self.loc()
        s = [f"{loc.title}", loc.desc]
        if loc.items: s.append("You see: " + ", ".join(it.name for it in loc.items))
        if loc.exits: s.append("Exits: " + ", ".join(loc.exits.keys()))
        return "\n".join(s)
    def inv(self) -> str:
        return "You are carrying nothing." if not self.inventory else "You carry: " + ", ".join(it.name for it in self.inventory)
    def game_over(self) -> bool: return self.dead or self.won

DIRECTIONS = {"n":"north","s":"south","e":"east","w":"west","u":"up","d":"down",
              "north":"north","south":"south","east":"east","west":"west","up":"up","down":"down","out":"out","in":"in"}

def tokenize(cmd: str) -> List[str]:
    return [t for t in cmd.strip().lower().replace(",", " ").split() if t]

class GoAction(Action):
    def matches(self, raw, tokens):
        return bool(tokens) and (tokens[0] in ("go","walk","run","move") or tokens[0] in DIRECTIONS)
    def run(self, game, raw, tokens):
        if tokens[0] in DIRECTIONS: return game.move(DIRECTIONS[tokens[0]])
        if len(tokens)>=2 and tokens[1] in DIRECTIONS: return game.move(DIRECTIONS[tokens[1]])
        return "Go where?"

class LookAction(Action):
    def matches(self, raw, tokens): return bool(tokens) and tokens[0] in ("look","l")
    def run(self, game, raw, tokens): return game.look()

class InvAction(Action):
    def matches(self, raw, tokens): return bool(tokens) and tokens[0] in ("inventory","inv","i")
    def run(self, game, raw, tokens): return game.inv()

class TakeAction(Action):
    def matches(self, raw, tokens): return bool(tokens) and tokens[0] in ("take","get","pick")
    def run(self, game, raw, tokens):
        if len(tokens)<2: return "Take what?"
        obj = tokens[-1]
        return game.take(obj)

class DropAction(Action):
    def matches(self, raw, tokens): return bool(tokens) and tokens[0] in ("drop","leave")
    def run(self, game, raw, tokens):
        if len(tokens)<2: return "Drop what?"
        return game.drop(tokens[1])

def play_cli(build_game_fn, intro="Type commands (e.g., 'go north', 'take key', 'look'). Type 'quit' to exit."):
    g = build_game_fn()
    print(intro); print(g.look())
    while not g.game_over():
        try:
            raw = input("> ")
        except EOFError:
            break
        if not raw: continue
        if raw.strip().lower() in ("quit","exit"): print("Goodbye."); break
        tokens = tokenize(raw)
        out = None
        for a in g.actions:
            if a.matches(raw, tokens):
                out = a.run(g, raw, tokens); break
        if out is None: out = "I don't understand."
        print(out)
        if g.game_over(): break
    if g.dead: print("You have died. Game over.")
    if g.won: print("You win!")
    return g

def run_scripted(build_game_fn, commands: List[str]) -> List[str]:
    g = build_game_fn(); outputs = [g.look()]
    for cmd in commands:
        if g.game_over(): break
        tokens = tokenize(cmd); out = None
        for a in g.actions:
            if a.matches(cmd, tokens):
                out = a.run(g, cmd, tokens); break
        if out is None: out = "I don't understand."
        outputs.append(f"> {cmd}\n{out}")
    if g.dead: outputs.append("You have died. Game over.")
    if g.won: outputs.append("You win!")
    return outputs


In [2]:

def build_action_castle():
    g = Game()
    L = {}
    def add(k,t,d): L[k]=g.add_loc(k,t,d)
    add("cottage","Cottage","A small cottage with a fishing pole here. Exits: out.")
    add("garden","Garden Path","A path runs north-south beside a rosebush.")
    add("pond","Fishing Pond","A tranquil pond. You might catch fish here.")
    add("wpath","Winding Path","A shady path leading north to a tall tree and east toward a bridge.")
    add("tree","Top of the Tall Tree","You can see far away. A sturdy branch is within reach.")
    add("draw","Drawbridge","A drawbridge leads east to the castle courtyard.")
    add("yard","Courtyard","A guard stands watch to the east toward the feasting hall.")
    add("tstairs","Tower Stairs","A stair spirals up to a locked tower door.")
    add("tower","Tower","The princess awaits above.")
    add("dstairs","Dungeon Stairs","A dark stairway goes down into the dungeon.")
    add("dungeon","Dungeon","A gloomy dungeon. You sense a ghost here; runes line the wall.")
    add("hall","Great Feasting Hall","Long tables stretch west to the courtyard and east to the throne room.")
    add("throne","Throne Room","A magnificent throne sits here.")
    # exits
    L["cottage"].exits.update({"out":"garden"})
    L["garden"].exits.update({"south":"pond","north":"wpath"})
    L["pond"].exits.update({"north":"garden"})
    L["wpath"].exits.update({"north":"tree","east":"draw","south":"garden"})
    L["tree"].exits.update({"down":"wpath"})
    L["draw"].exits.update({"east":"yard","west":"wpath"})
    L["yard"].exits.update({"west":"draw","east":"hall"})
    L["tstairs"].exits.update({"up":"tower","down":"yard"})
    L["tower"].exits.update({"down":"tstairs"})
    L["dstairs"].exits.update({"down":"dungeon","up":"hall"})
    L["dungeon"].exits.update({"up":"dstairs"})
    L["hall"].exits.update({"west":"yard","east":"throne","down":"dstairs","up":"tstairs"})
    L["throne"].exits.update({"west":"hall"})
    g.player_loc = "cottage"

    # items
    g.add_item("pole", aliases=["fishing","fishingpole","fishing-pole","rod"], desc="A simple fishing pole.")
    g.add_item("rosebush", portable=False, desc="A lovely rosebush.")
    g.add_item("rose", desc="A fragrant red rose.")
    g.add_item("branch", aliases=["club","stick"], desc="A sturdy branch that makes a decent club.")
    g.add_item("fish", desc="A freshly caught fish.")
    g.add_item("key", desc="A small brass key.")
    g.add_item("candle", desc="A wax candle.")
    g.add_item("lamp", aliases=["lantern"], desc="An oil lamp.")
    g.add_item("crown", desc="A golden crown.")
    g.add_item("troll", portable=False, desc="A hungry troll lurks here.")
    g.add_item("guard", portable=False, desc="A surly guard blocks the way east.")
    g.add_item("princess", portable=False, desc="The princess awaits suitors.")

    # place items (rose explicitly in garden)
    g.put_item("pole","cottage")
    g.put_item("rosebush","garden")
    g.put_item("rose","garden")
    g.put_item("troll","draw")
    g.put_item("guard","yard")
    g.put_item("key","hall")
    g.put_item("candle","hall")
    g.put_item("lamp","dstairs")
    g.put_item("crown","dungeon")
    g.put_item("princess","tower")

    # blocks
    L["yard"].blocks.append(Block("yard","east","The guard bars your way.", lambda game: "guard_stunned" in game.flags))
    L["dstairs"].blocks.append(Block("dstairs","down","It's too dark to go down.", lambda game: "light" in game.flags))
    L["tstairs"].blocks.append(Block("tstairs","up","The tower door is locked.", lambda game: "tower_unlocked" in game.flags))

    # actions
    g.actions.extend([GoAction(), LookAction(), InvAction(), TakeAction(), DropAction()])

    class Catch(Action):
        def matches(self, raw, tokens): return tokens and tokens[0] in ("catch","fish")
        def run(self, game, raw, tokens):
            if game.loc().key!="pond": return "You can't catch fish here."
            if "pole" not in game.get_inventory_names(): return "You need a pole."
            if "fish" in game.get_inventory_names(): return "You already have a fish."
            game.inventory.append(game.items["fish"]); return "You catch a fish!"
    g.actions.append(Catch())

    class GetBranch(Action):
        def matches(self, raw, tokens):
            return tokens and ((tokens[0] in ("get","grab","take")) and (tokens[-1] in ("branch","stick","club")))
        def run(self, game, raw, tokens):
            if game.loc().key!="tree": return "You can't reach a branch here."
            if "branch" in game.get_inventory_names(): return "You already have a branch."
            game.inventory.append(game.items["branch"]); return "You break off a sturdy branch."
    g.actions.append(GetBranch())

    class Give(Action):
        def matches(self, raw, tokens): return tokens and tokens[0]=="give"
        def run(self, game, raw, tokens):
            if "fish" not in game.get_inventory_names(): return "You have no fish to give."
            if game.loc().key!="draw": return "No one here wants that."
            game.flags.add("troll_fed"); game.inventory = [it for it in game.inventory if it.name!="fish"]
            return "The troll happily devours the fish and lets you pass."
    g.actions.append(Give())

    class Hit(Action):
        def matches(self, raw, tokens): return tokens and tokens[0] in ("hit","attack","smack")
        def run(self, game, raw, tokens):
            if game.loc().key!="yard": return "There's nothing to hit here."
            if "branch" not in game.get_inventory_names(): return "With what?"
            game.flags.add("guard_stunned"); return "You bop the guard with the branch. He slumps over. The way east is clear."
    g.actions.append(Hit())

    class Unlock(Action):
        def matches(self, raw, tokens): return tokens and tokens[0]=="unlock"
        def run(self, game, raw, tokens):
            if "key" not in game.get_inventory_names(): return "You need a key."
            game.flags.add("tower_unlocked"); return "You unlock the tower door."
    g.actions.append(Unlock())

    class Light(Action):
        def matches(self, raw, tokens): return tokens and tokens[0]=="light"
        def run(self, game, raw, tokens):
            if len(tokens)<2: return "Light what?"
            target = tokens[1]
            if target in ("lamp","lantern","candle"):
                if target not in game.get_inventory_names(): return f"You don't have a {target}."
                game.flags.add("light"); return f"You light the {target}."
            return "That won't light."
    g.actions.append(Light())

    class Read(Action):
        def matches(self, raw, tokens): return tokens and tokens[0]=="read"
        def run(self, game, raw, tokens):
            if game.loc().key!="dungeon": return "There's nothing to read here."
            game.flags.add("ghost_banished"); return "You read the runes aloud. A wail echoes as the ghost vanishes."
    g.actions.append(Read())

    class Wear(Action):
        def matches(self, raw, tokens): return tokens and tokens[0]=="wear"
        def run(self, game, raw, tokens):
            if "crown" not in game.get_inventory_names(): return "You have no crown."
            game.flags.add("wearing_crown"); return "You don the crown. You feel... royal."
    g.actions.append(Wear())

    class RoseAction(Action):
        def matches(self, raw, tokens):
            return tokens and (tokens[0] in ("pick","take","get") and tokens[-1]=="rose")
        def run(self, game, raw, tokens):
            if game.loc().key!="garden": return "There are no roses to pick here."
            if "rose" in game.get_inventory_names(): return "You already have a rose."
            here = game.loc()
            rose_here = None
            for it in here.items:
                if it.name=="rose": rose_here = it; break
            if not rose_here:
                rose_here = game.items["rose"]; here.items.append(rose_here)
            here.items.remove(rose_here); game.inventory.append(rose_here)
            game.flags.add("rose_picked"); return "You pick a beautiful rose."
    g.actions.append(RoseAction())

    class Smell(Action):
        def matches(self, raw, tokens): return tokens and tokens[0]=="smell"
        def run(self, game, raw, tokens):
            if "rose" in game.get_inventory_names(): game.flags.add("smelled_rose"); return "Mmm. Lovely scent."
            return "You don't have a rose."
    g.actions.append(Smell())

    class Propose(Action):
        def matches(self, raw, tokens): return tokens and tokens[0] in ("propose","marry","marriage")
        def run(self, game, raw, tokens):
            if game.loc().key!="tower": return "No one here to propose to."
            if "rose" not in game.get_inventory_names():
                return "Perhaps bring a rose? (Hint: pick/take/get rose in the Garden.)"
            game.flags.add("royal"); return "The princess accepts! You are now betrothed."
    g.actions.append(Propose())

    class Sit(Action):
        def matches(self, raw, tokens): return tokens and tokens[0]=="sit"
        def run(self, game, raw, tokens):
            if game.loc().key!="throne": return "There's no throne here."
            if "royal" in game.flags and "wearing_crown" in game.flags:
                game.won=True; return "You sit upon the throne as rightful ruler."
            return "You do not yet meet the requirements to claim the throne."
    g.actions.append(Sit())

    return g

# auto-win smoke test
auto = "take pole, out, south, catch fish, north, pick rose, smell rose, north, up, get branch, down, east, give fish, east, hit guard with branch, east, get candle, west, down, get lamp, light lamp, down, light candle, read runes, get crown, up, up, up, unlock door, up, propose, wear crown, down, down, east, east, sit"
cmds = [t.strip().replace('  ',' ') for t in auto.split(",")]
print(run_scripted(build_action_castle, cmds)[-1])


> sit
There's no throne here.


### Interactive play
Uncomment and run: `play_cli(build_action_castle)`

In [4]:
play_cli(build_action_castle)

Type commands (e.g., 'go north', 'take key', 'look'). Type 'quit' to exit.
Cottage
A small cottage with a fishing pole here. Exits: out.
You see: pole
Exits: out
> take pole
Taken pole.
> go out
Garden Path
A path runs north-south beside a rosebush.
You see: rosebush, rose
Exits: south, north
> go south
Fishing Pond
A tranquil pond. You might catch fish here.
Exits: north
> catch fish
You catch a fish!
> go north
Garden Path
A path runs north-south beside a rosebush.
You see: rosebush, rose
Exits: south, north
> pick rose
Taken rose.
> smell rose
Mmm. Lovely scent.
> go north
Winding Path
A shady path leading north to a tall tree and east toward a bridge.
Exits: north, east, south
> go up
You can't go that way.
> go north
Top of the Tall Tree
You can see far away. A sturdy branch is within reach.
Exits: down
> get branch
You don't see that here.
> grab branch
You break off a sturdy branch.
> go down
Winding Path
A shady path leading north to a tall tree and east toward a bridge.
Exits: